In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**feature_Bow**

In [ ]:
import json
import pandas as pd
identification_csv = pd.read_csv("../input/data-provided/data_identification.csv")
emotion_csv = pd.read_csv("../input/data-provided/emotion.csv")
with open('../input/tweets/tweets_DM.json', 'r') as tweet_data:
    values = [json.loads(line)['_source']['tweet'] for line in tweet_data.readlines() ]
    




In [ ]:
raw_data = pd.DataFrame(values)
raw_data = raw_data.sort_values(by=['tweet_id'])
identification_csv = identification_csv.sort_values(by=['tweet_id'])
ident_dict = identification_csv.set_index('tweet_id')['identification'].to_dict()
raw_data['identification'] = ident_dict.values()


In [ ]:
df_train = raw_data.loc[lambda raw_data: raw_data['identification'] == 'train',:]
df_test = raw_data.loc[lambda raw_data: raw_data['identification'] == 'test',:]

df_train = df_train.sort_values(by=['tweet_id'])
emotion_csv = emotion_csv.sort_values(by=['tweet_id'])
emotion_dict = emotion_csv.set_index('tweet_id')['emotion'].to_dict()
df_train['emotion'] = emotion_dict.values()
print(df_train.shape)
df_train

In [ ]:
## save as pickle file
df_train.to_pickle("/kaggle/working/train_df.pkl") 
df_test.to_pickle("/kaggle/working/test_df.pkl") 
df_train

In [ ]:
import pandas as pd
# load a pickle file
train_df = pd.read_pickle("../input/pickle-data/train_df.pkl")[:-1000]
test_df =  pd.read_pickle("../input/pickle-data/train_df.pkl")[-1000:]
final_test_df = pd.read_pickle("../input/pickle-data/test_df.pkl")
final_test_df.shape
final_test_df.head()
print(train_df.shape)
train_df.dropna(inplace=True)
print(train_df.shape)
print(test_df.shape)
test_df.dropna(inplace=True)
print(test_df.shape)
train_df

In [ ]:
### generate feature_Bow ###

from sklearn.feature_extraction.text import CountVectorizer
import nltk

# build analyzers (bag-of-words)
BOW_500 = CountVectorizer(max_features=1000, tokenizer=nltk.word_tokenize) 

# apply analyzer to training data
BOW_500.fit(train_df['text'][:20000])

train_data_BOW_features_500 = BOW_500.transform(train_df['text'][:20000])

## check dimension
train_data_BOW_features_500.shape

In [ ]:
### generate X_train X_test ###

# for a classificaiton problem, you need to provide both training & testing data
X_train = BOW_500.transform(train_df['text'])
y_train = train_df['emotion']

X_test = BOW_500.transform(test_df['text'])
y_test = test_df['emotion']

## take a look at data dimension is a good habbit  :)
print('X_train.shape: ', X_train.shape)
print('y_train.shape: ', y_train.shape)
print('X_test.shape: ', X_test.shape)
print('y_test.shape: ', y_test.shape)

In [ ]:
### build Desicion Model ###
from sklearn.tree import DecisionTreeClassifier

DT_model = DecisionTreeClassifier(random_state=0)

## training!
DT_model = DT_model.fit(X_train, y_train)

## predict!
y_train_pred = DT_model.predict(X_train)
y_test_pred = DT_model.predict(X_test)

## so we get the pred result
y_test_pred[:10]

In [ ]:
### accuracy ###
from sklearn.metrics import accuracy_score

acc_train = accuracy_score(y_true=y_train, y_pred=y_train_pred)
acc_test = accuracy_score(y_true=y_test, y_pred=y_test_pred)

print('training accuracy: {}'.format(round(acc_train, 2)))
print('testing accuracy: {}'.format(round(acc_test, 2)))


## precision, recall, f1-score,
from sklearn.metrics import classification_report

print(classification_report(y_true=y_test, y_pred=y_test_pred))

In [ ]:
X_train = BOW_500.transform(final_test_df['text'])
final_train_pred = DT_model.predict(X_train)

final_test_df['emotion'] = final_train_pred
csvData = final_test_df[['tweet_id','emotion']].to_csv('/kaggle/working/predict.csv',header=['id','emotion'],index=False) 
final_train_pred